In [1]:
import parmed
import numpy as np
import itertools

from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

from sys import stdout
import sys

# Load protein system:

In [2]:
#Load up CHARMM-GUI pdb, and create a PARMED structure from it.  
prot_pdb = PDBFile('./1xep-processed.pdb')
omm_forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')
prot_system = omm_forcefield.createSystem(prot_pdb.topology, rigidWater=False)
prot_structure = parmed.openmm.load_topology(prot_pdb.topology,
                                           prot_system,
                                           xyz=prot_pdb.positions)

# Load drug system:

In [3]:
#Load up the parameterized drug system, and again make it into a parmed structure:
drug_system = XmlSerializer.deserialize(open('./drug_system.xml').read())
drug_pdbfile = PDBFile('./catechol_aligned.pdb')

drug_structure = parmed.openmm.load_topology(drug_pdbfile.topology,
                                                drug_system,
                                                xyz=drug_pdbfile.positions)

# Combine:

In [4]:
#This is the biggest step but it takes 1 second:
complex_structure = prot_structure + drug_structure


# Turn back into an openmm system

In [5]:
#set periodic boundary conditons (you get these from the processed pdb file, 1xep-processed.pdb)
#64.871   64.871   64.871  60.00  60.00  90.00 P 1           1
#note parmed (and PBD files) uses angstrom by default. 
complex_structure.box = (64.871, 64.871, 64.871, 60, 60, 90)

#Turn into an OpenMM System object for simulations:
#These settings will be stuck unless you re-run this script! Luckily they're pretty standard settings.
complex_system = complex_structure.createSystem(nonbondedMethod=PME,
                                                nonbondedCutoff=0.9*nanometer,
                                               constraints=HBonds,
                                                rigidWater=True)

In [6]:
complex_structure.save('complex_plus_water.parm7')

# Save output as PDB, PSF, and serialized openmm system

In [7]:
#Save output:
complex_structure.save('./complex_coords.pdb', overwrite=True)

#PSF files don't like having numbered atom types, because at 10,000 VMD fails
#So just set them all to zero.
for a in complex_structure.atoms:
    a.type = '0'
complex_structure.save('./complex_struct_.psf', overwrite=True)

with open('./complex_system_.xml', 'w') as f:
    f.write(
            XmlSerializer.serialize(
                complex_system
            )
    )